In [9]:
# Python 3 project to predict the outcome of the 2023 FIFA Women's World Cup

# Library Imports
import numpy as np
import pandas as pd
import re

In [10]:
# Read in the data
game_data = pd.read_csv("data/results.csv")

In [11]:
# Clean the data 
# This will tell me the number of items in each column that has an NA value
game_data.isna().sum()

date          0
home_team     0
away_team     0
home_score    0
away_score    0
tournament    0
city          0
country       0
neutral       0
dtype: int64

In [12]:
game_data.dtypes

date          object
home_team     object
away_team     object
home_score     int64
away_score     int64
tournament    object
city          object
country       object
neutral         bool
dtype: object

In [13]:
game_data["date"] = pd.to_datetime(game_data["date"])
game_data.dtypes

date          datetime64[ns]
home_team             object
away_team             object
home_score             int64
away_score             int64
tournament            object
city                  object
country               object
neutral                 bool
dtype: object

In [14]:
#Sort values by date and display the most recent 5 games in the notebook 
game_data.sort_values("date").tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
4879,2022-07-22,Sweden,Belgium,1,0,UEFA Euro,Leigh,Sweden,True
4880,2022-07-23,France,Netherlands,1,0,UEFA Euro,Rotherham,France,True
4881,2022-07-26,England,Sweden,4,0,UEFA Euro,Sheffield,England,False
4882,2022-07-27,Germany,France,2,1,UEFA Euro,Milton Keynes,Germany,True
4883,2022-07-31,England,Germany,2,1,UEFA Euro,London,England,False


In [19]:
# Do the same for the shootouts data
# Import
shootout_data = pd.read_csv("data/shootouts.csv")
# Find NA data
shootout_data.isna().sum()
# Change date type
shootout_data["date"] = pd.to_datetime(shootout_data["date"])
shootout_data.dtypes
#Sort values by date and display the most recent 5 games in the notebook 
shootout_data.sort_values("date").tail()

,date,home_team,away_team,winner
21,2021-10-26,Ethiopia,Uganda,Uganda
22,2022-01-30,Chinese Taipei,Philippines,Philippines
23,2022-02-03,China PR,Japan,China PR
24,2022-02-22,Mali,Senegal,Senegal
25,2022-02-23,Sweden,Italy,Sweden
